In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
#因下載的資料分兩部分，因此先個別檢查
red_wine = pd.read_csv('Wine_quality/winequality-red.csv', sep=';', header=0)
white_wine = pd.read_csv('Wine_quality/winequality-white.csv', sep=';', header=0)
print(red_wine)
print(white_wine)
print(red_wine.shape)
print(white_wine.shape)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
5               7.4             0.660         0.00             1.8      0.075   
6               7.9             0.600         0.06             1.6      0.069   
7               7.3             0.650         0.00             1.2      0.065   
8               7.8             0.580         0.02             2.0      0.073   
9               7.5             0.500         0.36             6.1      0.071   
10              6.7             0.580         0.08             1.8      0.097   
11              7.5         

In [3]:
#將這兩個csv檔個先增加一欄位'type'(紅酒為1,白酒為0)省去one hot encoding的作業
red_wine['type'] = pd.Series((1), index=red_wine.index) 
white_wine['type'] = pd.Series((0), index=white_wine.index)
#檢查是否有誤
print(red_wine)
print(white_wine)
print(red_wine.shape)
print(white_wine.shape)

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
5               7.4             0.660         0.00             1.8      0.075   
6               7.9             0.600         0.06             1.6      0.069   
7               7.3             0.650         0.00             1.2      0.065   
8               7.8             0.580         0.02             2.0      0.073   
9               7.5             0.500         0.36             6.1      0.071   
10              6.7             0.580         0.08             1.8      0.097   
11              7.5         

In [4]:
#把兩個csv做合併
all_wine=pd.concat([red_wine,white_wine],axis=0)
#並將資料打亂
all_wine=all_wine.sample(frac=1).reset_index(drop=True)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
#數據切片，最後一欄為'type'，切成 X 和 y
X = all_wine.iloc[:,:-1]
y = all_wine.iloc[:,-1:]
#建立 訓練及測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3 , random_state=42)


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
#使用SVC模型
svm=SVC()
#交叉驗證
score=cross_val_score(svm,X,y,cv=5,scoring='accuracy')
print(np.mean(score))

0.9493597308603376


In [7]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#餵數據並看結果如何(未經過正規化)
svm_clf = svm.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)
svm_clf.score(X_test, y_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy:', accuracy_score(y_test, y_pred))

[[1440   16]
 [  88  406]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      1456
           1       0.96      0.82      0.89       494

   micro avg       0.95      0.95      0.95      1950
   macro avg       0.95      0.91      0.93      1950
weighted avg       0.95      0.95      0.95      1950

accuracy: 0.9466666666666667


In [8]:
#餵數據並看結果如何(經過正規化)
X_train = scale(X_train)
X_test = scale(X_test)
svm_clf = svm.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print('accuracy:', accuracy_score(y_test, y_pred))

[[1453    3]
 [   5  489]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1456
           1       0.99      0.99      0.99       494

   micro avg       1.00      1.00      1.00      1950
   macro avg       1.00      0.99      0.99      1950
weighted avg       1.00      1.00      1.00      1950

accuracy: 0.9958974358974358


In [9]:
X_test

array([[-0.62057743, -0.07479743, -0.03746501, ...,  0.84008145,
        -0.99879822,  0.21280922],
       [-0.8497318 ,  0.10354574, -0.31367207, ...,  1.1692802 ,
         0.83102855,  1.36552586],
       [-0.00949913, -0.43148376, -0.03746501, ..., -1.33263031,
         1.82911589,  0.21280922],
       ...,
       [ 0.29604002,  0.8169184 ,  0.37684558, ..., -0.93759181,
         0.74785461, -2.09262404],
       [-1.07888616,  3.96764766,  2.37934676, ...,  0.44504295,
         0.83102855,  0.21280922],
       [ 0.67796395,  1.20332859, -0.93513795, ...,  0.3792032 ,
        -1.24832005,  0.21280922]])

經過正規化後，直接提高5%左右的accuracy